In [1]:
from bs4 import BeautifulSoup
import requests
import csv
from collections import namedtuple
import googlemaps


In [2]:
url='https://pt.wikipedia.org/wiki/Lista_de_universidades_federais_do_Brasil'
#URL da página que vamos utilizar para puxar as informações das faculdades.

In [3]:
r=requests.get(url)
#Aqui usando a biblioteca requests, vamos fazer uma requisição para o link da Wikipedia

In [4]:
soup = BeautifulSoup(r.text, 'html.parser')
#Pegamos todo o arquivo HTML


In [5]:
tabela=soup.find('tbody')
#Aqui pego somente a parte que importa, no caso a tbody que é a que traz a lista.

In [6]:
rows=tabela.find_all('tr')

In [7]:
columns=['num','Região',
 'Unidade_federativa',
 'Nome',
 'Sigla',
 'Discentes_de_graduação',
 'Discentes_de_pós_graduação',
 'Fundação' ]

In [8]:
def lertudo():
    UF = namedtuple('UF',columns)
    for i in range(1,len(rows)):
        tds=rows[i].find_all('td')
        campo=([td.text.replace('\n','').replace('\xa0','') for td in tds])
        yield UF(*campo)
            
#Aqui faço a leitura dos campos e faço o armazenamento dos dados numa namedtuple.

it = list( [(d.num, d.Região,d.Unidade_federativa,d.Nome,d.Sigla,d.Discentes_de_graduação,d.Discentes_de_pós_graduação,d.Fundação)
         for d in lertudo()])   

In [9]:
def gerar_rest(lat,long):
    lista=[lat,long]
    print(lista)
    for i in range(len(lista)):
        places_result  = gmaps.places_nearby(location=lista, radius = 100, open_now =False , type = 'restaurant')
        for place in places_result['results']:
            my_place_id = place['place_id']
            my_fields = ['name']
            places_details  = gmaps.place(place_id= my_place_id , fields= my_fields)
            print(places_details['result'])
#Aqui, por mais que não tenha dado certo iria trazer os restaurantes próximos, a ideia para mostrar a distância entre o restaurante 
#e a faculdade seria usar a API distancematrix, e faria uma requisição passando a talvez o nome da faculdade e o nome do restaurante
#ou a lat, long da faculdade e a do restaurante.

In [10]:
def gerar_end(univ):
    enderecos=[]
    params={'key':'KEY_API_HERE','address':univ}
    base_url='https://maps.googleapis.com/maps/api/geocode/json?'
    response=requests.get(base_url,params=params).json() #Fazendo a requisição a API.
    response.keys()
    if response['status']=='OK':
        geometry=response['results'][0]['geometry']
        enderecos.append([geometry['location']['lat'],geometry['location']['lng'],response['results'][0]['formatted_address']])
        
    return enderecos

#Essa parte, como na outra recebe uma lista e faz uma resquisição a API geocode, e me retorna os dados como lat,long e endereço completo
#da faculdade, os dados são armazenados numa lista.

In [11]:
def arrumar(lat,long,end,_all,i):
    lat=str(lat).replace('(','').replace(')','')
    long=str(long).replace('(','').replace(')','')
    end=str(end).replace('(','').replace(')','')
    wor=(str(it[i]).strip('[]').replace('(','').replace(')','').replace('"',''),lat,long,end)
    #Aqui é tão somente para tirar alguns caractéres das palavras.
    return wor

In [12]:
def gerar_csv():
    arq = open('Atividade_Final.csv', 'w', newline='', encoding='iso-8859-1')
    #Abertura do arquivo CSV (Que sempre vai sobrescrever)
    obj = csv.writer(arq)
    head=['Número','Região','Unidade Federativa','Nome','Sigla','Discentes de graduação','Discentes de pós graduação','Fundação','Lat','Long','End' ]
    #Aqui eu salvo o cabeçario das colunas.
    obj.writerow(head)
    i=0
    for u in lertudo():
        #Aqui faço a abertura da namedtuple.
        lat,long,end=(zip(*gerar_end(u.Nome)))
        #Aqui chamo a função que me mostra os endereços e passo como argumento a namedtuple UF com os nomes da faculdades.
        #Isso me gera um arquivo compactado, faço uso do zip* para descompactar e colocar nas respectivas variáveis
        obj.writerow(arrumar(lat,long,end,it,i))
        i=i+1
        #Escrevo os dados que foram gerados acima, mas antes passo pela função arrumar que vai fazer uma limpeza nos dados.
    
    arq.close()
    #O arquivo então é encerrado e a mensagem aparece.
    print("Exportação concluida.")

In [13]:
gerar_csv()

Exportação concluida.
